Importing libraries I will need for this project

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda update pandas # updating pandas 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge

The following packages will be UPDATED:

    certifi:         2019.9.11-py36_0  --> 2019.9.11-py36_0     conda-forge

The following packages will be DOWNGRADED:

    ca-certificates: 2019.10.16-0      --> 2019.9.11-hecc5488_0 conda-forge
    openssl:         1.1.1d-h7b6447c_3 --> 1.1.1d-h516909a_0    conda-forge


certifi-2019.9.11    | 147 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/

In [3]:
pd.__version__

'0.25.3'

# MADRID

### 1. Creating dataframe for Madrid

Reading Wikipedia file of Madrids' districts and neighbourhoods to dataframe

In [4]:
madrid_df=pd.read_html("https://es.wikipedia.org/wiki/Anexo:Distritos_de_Madrid")[0]
madrid_df.head()

,Número,Nombre,Superficie[n. 1]​ (Ha.),Población,Densidad(Hab./Ha.),Imagen,Barrios
0,1.0,Centro,NaN,NaN,NaN,NaN,Palacio (11)Embajadores (12)Cortes (13)Justici...
1,2.0,Arganzuela,NaN,NaN,NaN,NaN,Imperial (21)Acacias (22)Chopera (23)Legazpi (...
2,3.0,Retiro,NaN,NaN,NaN,NaN,Pacífico (31)Adelfas (32)Estrella (33)Ibiza (3...
3,4.0,Salamanca,NaN,NaN,NaN,NaN,Recoletos (41)Goya (42)Fuente del Berro (43)Gu...
4,5.0,Chamartín,NaN,NaN,NaN,NaN,El Viso (51)Prosperidad (52)Ciudad Jardín (53)...


Dropping unnecessary columns and renaming columns I will work with (translating to English)

In [5]:
madrid_df.drop(['Número', 'Superficie[n. 1]​ (Ha.)', 'Población', 'Densidad(Hab./Ha.)', 'Imagen'], axis=1, inplace=True)
madrid_df.rename(columns={'Nombre': 'District', 'Barrios': 'Neighbourhood'}, inplace=True)
madrid_df.head(7)

,District,Neighbourhood
0,Centro,Palacio (11)Embajadores (12)Cortes (13)Justici...
1,Arganzuela,Imperial (21)Acacias (22)Chopera (23)Legazpi (...
2,Retiro,Pacífico (31)Adelfas (32)Estrella (33)Ibiza (3...
3,Salamanca,Recoletos (41)Goya (42)Fuente del Berro (43)Gu...
4,Chamartín,El Viso (51)Prosperidad (52)Ciudad Jardín (53)...
5,Tetuán,Bellas Vistas (61)Cuatro Caminos (62)Castillej...
6,Chamberí,Gaztambide (71)Arapiles (72)Trafalgar (73)Alma...


Selecting first 7 rows/districts we will work with

In [6]:
madrid_df=madrid_df.iloc[0:7]
madrid_df

,District,Neighbourhood
0,Centro,Palacio (11)Embajadores (12)Cortes (13)Justici...
1,Arganzuela,Imperial (21)Acacias (22)Chopera (23)Legazpi (...
2,Retiro,Pacífico (31)Adelfas (32)Estrella (33)Ibiza (3...
3,Salamanca,Recoletos (41)Goya (42)Fuente del Berro (43)Gu...
4,Chamartín,El Viso (51)Prosperidad (52)Ciudad Jardín (53)...
5,Tetuán,Bellas Vistas (61)Cuatro Caminos (62)Castillej...
6,Chamberí,Gaztambide (71)Arapiles (72)Trafalgar (73)Alma...


Exploding dataframe to get a new row for each neighbourhood in a district

In [7]:
madrid_df=madrid_df.assign(Neighbourhood=madrid_df.Neighbourhood.str.split(')')).explode('Neighbourhood').reset_index(drop=True)
madrid_df.head(7)

,District,Neighbourhood
0,Centro,Palacio (11
1,Centro,Embajadores (12
2,Centro,Cortes (13
3,Centro,Justicia (14
4,Centro,Universidad (15
5,Centro,Sol (16
6,Centro,


Dropping rows with null values and resetting indices

In [8]:
madrid_df = madrid_df.drop([6,14,21,28,35,42,49])
madrid_df = madrid_df.reset_index(drop=True)
madrid_df

,District,Neighbourhood
0,Centro,Palacio (11
1,Centro,Embajadores (12
2,Centro,Cortes (13
3,Centro,Justicia (14
4,Centro,Universidad (15
5,Centro,Sol (16
6,Arganzuela,Imperial (21
7,Arganzuela,Acacias (22
8,Arganzuela,Chopera (23
9,Arganzuela,Legazpi (24


In [68]:
madrid_df.shape

(43, 2)

Getting latitude and longitude values for each neighbourhood. I couldn't create an automated process, so I'll do this manually (maybe you can suggest something to deal with this problem faster?) 

In [2]:
address = 'Madrid, Centro, Palacio'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Palacio are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Palacio are 40.41512925, -3.71561799839909.


In [30]:
address = 'Madrid, Embajadores'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Embajadores are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Embajadores are 40.40968055, -3.70164442641322.


In [32]:
address = 'Madrid, Centro, Cortes'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Cortes are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Cortes are 40.4143476, -3.69852518277385.


In [73]:
address = 'Madrid, Justicia'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Justicia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Justicia are 40.4239569, -3.69574732085505.


In [74]:
address = 'Madrid, Universidad'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Universidad are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Universidad are 40.4252644, -3.70660596905568.


In [36]:
address = 'Madrid, Sol'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Sol are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Sol are 40.4178272, -3.70222208991507.


In [75]:
address = 'Madrid, Imperial'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Imperial are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Imperial are 40.4069151, -3.71732906254996.


In [76]:
address = 'Madrid, Acacias'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Acacias are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Acacias are 40.4040749, -3.7059572.


In [81]:
address = 'Madrid, Arganzuela, Chopera'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Chopera are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Chopera are 40.39489315, -3.69970511346301.


In [82]:
address = 'Madrid, Arganzuela, Legazpi'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Legazpi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Legazpi are 40.3911717, -3.6951902.


In [83]:
address = 'Madrid, Arganzuela, Delicias'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Delicias are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Delicias are 40.39729215, -3.68949484969473.


In [84]:
address = 'Madrid, Arganzuela, Palos de Moguer'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Palos de Moguer are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Palos de Moguer are 40.40392685, -3.69556132795742.


In [85]:
address = 'Madrid, Arganzuela, Atocha'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Atocha are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Atocha are 40.40068665, -3.68239115386921.


In [86]:
address = 'Madrid, Retiro, Pacífico'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Pacífico are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Pacífico are 40.40336395, -3.67897986675935.


In [87]:
address = 'Madrid, Retiro, Adelfas'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Adelfas are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Adelfas are 40.40027955, -3.67176728223261.


In [88]:
address = 'Madrid, Retiro, Estrella'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Estrella are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Estrella are 40.41441435, -3.66521789344363.


In [89]:
address = 'Madrid, Retiro, Ibiza'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Ibiza are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Ibiza are 40.41891775, -3.67409049246932.


In [90]:
address = 'Madrid, Retiro, Jerónimos'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Jerónimos are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Jerónimos are 40.4137286, -3.68535643549675.


In [91]:
address = 'Madrid, Retiro, Niño Jesús'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Niño Jesús are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Niño Jesús are 40.4134414, -3.67333679089834.


In [92]:
address = 'Madrid, Salamanca, Recoletos'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Recoletos are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Recoletos are 40.42417315, -3.68548726497414.


In [93]:
address = 'Madrid, Salamanca, Goya'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Goya are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Goya are 40.4256052, -3.6743653201567.


In [94]:
address = 'Madrid, Salamanca, Fuente del Berro'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Fuente del Berro are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Fuente del Berro are 40.42513105, -3.66423759698396.


In [95]:
address = 'Madrid, Salamanca, Guindalera'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Guindalera are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Guindalera are 40.4362387, -3.66796955160589.


In [96]:
address = 'Madrid, Salamanca, Lista'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Lista are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Lista are 40.43119915, -3.67550565570494.


In [97]:
address = 'Madrid, Salamanca, Castellana'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Castellana are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Castellana are 40.43382275, -3.68400358245116.


In [111]:
address = 'Madrid, Chamartín'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, El Viso are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, El Viso are 40.4607638, -3.67753405826461.


In [112]:
address = 'Madrid, Chamartín, Prosperidad'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Prosperidad are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Prosperidad are 40.4437817, -3.6750252.


In [113]:
address = 'Madrid, Chamartín, Ciudad Jardín'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Ciudad Jardín are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Ciudad Jardín are 40.456774, -3.6704858.


In [119]:
address = 'Madrid, Chamartin'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Hispanoamérica are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Hispanoamérica are 40.4607638, -3.67753405826461.


In [123]:
address = 'Madrid, Chamartín'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Nueva España are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Nueva España are 40.4607638, -3.67753405826461.


In [124]:
address = 'Madrid, Chamartín, Castilla'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Castilla are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Castilla are 40.4370393, -3.6773899.


In [125]:
address = 'Madrid, Tetuán, Bellas Vistas'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Bellas Vistas are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Bellas Vistas are 40.45226305, -3.70744452722191.


In [126]:
address = 'Madrid, Tetuán, Cuatro Caminos '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Cuatro Caminos  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Cuatro Caminos  are 40.4470709, -3.7042421.


In [127]:
address = 'Madrid, Tetuán, Castillejos '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Castillejos  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Castillejos  are 40.4558722, -3.7079658.


In [128]:
address = 'Madrid, Tetuán, Almenara '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Almenara  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Almenara  are 40.4711927, -3.69208516102331.


In [129]:
address = 'Madrid, Tetuán, Valdeacederas '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Valdeacederas  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Valdeacederas  are 40.4645454, -3.6950053.


In [130]:
address = 'Madrid, Tetuán, Berruguete '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Berruguete  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Berruguete  are 40.4588005, -3.7048308.


In [131]:
address = 'Madrid, Chamberí, Gaztambide '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Gaztambide  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Gaztambide  are 40.43467955, -3.71490270979186.


In [132]:
address = 'Madrid, Chamberí, Arapiles '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Arapiles  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Arapiles  are 40.43418655, -3.70792253324742.


In [133]:
address = 'Madrid, Chamberí, Trafalgar '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Trafalgar  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Trafalgar  are 40.4332491, -3.70124589579836.


In [134]:
address = 'Madrid, Chamberí, Almagro '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Almagro  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Almagro  are 40.4313814, -3.69301420454145.


In [135]:
address = 'Madrid, Chamberí, Ríos Rosas '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Ríos Rosas  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Ríos Rosas  are 40.44279905, -3.69783517044679.


In [136]:
address = 'Madrid, Chamberí, Vallehermoso '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid, Vallehermoso  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid, Vallehermoso  are 40.4430572, -3.71168099208445.


Adding latitude and longitude columns to our table of districts and neighbourhoods of Madrid.

In [9]:
m_latitude = [40.41512925, 40.40968055, 40.4143476, 40.4239569, 40.4252644, 40.4178272, 40.4069151, 40.4040749, 40.39489315, 40.3911717, 
             40.39729215, 40.40392685, 40.40068665, 40.40336395, 40.40027955, 40.41441435, 40.41891775, 40.4137286, 40.4134414, 40.42417315,
             40.4256052, 40.42513105, 40.4362387, 40.43119915, 40.43382275, 40.445372, 40.4437817, 40.456774, 40.455297, 40.463031, 40.4370393,
             40.45226305, 40.4470709, 40.4558722, 40.4711927, 40.4645454, 40.4588005, 40.43467955, 40.43418655, 40.4332491, 40.4313814, 40.44279905,
             40.4430572]
m_longitude = [-3.71561799839909, -3.70164442641322, -3.69852518277385, -3.69574732085505, -3.70660596905568, -3.70222208991507, -3.71732906254996,
              -3.7059572, -3.69970511346301, -3.6951902, -3.68949484969473, -3.69556132795742, -3.68239115386921, -3.67897986675935, -3.67176728223261,
              -3.66521789344363, -3.67409049246932, -3.68535643549675, -3.67333679089834, -3.68548726497414, -3.6743653201567, -3.66423759698396,
              -3.66796955160589, -3.67550565570494, -3.68400358245116, -3.685085, -3.6750252, -3.6704858, -3.675684, -3.677894, -3.6773899,
              -3.70744452722191, -3.7042421, -3.7079658, -3.69208516102331, -3.6950053, -3.7048308, -3.71490270979186, -3.70792253324742,
              -3.70124589579836, -3.69301420454145, -3.69783517044679, -3.71168099208445]
madrid_df['Latitude'] = m_latitude 
madrid_df['Longitude'] = m_longitude
madrid_df

,District,Neighbourhood,Latitude,Longitude
0,Centro,Palacio (11,40.415129,-3.715618
1,Centro,Embajadores (12,40.409681,-3.701644
2,Centro,Cortes (13,40.414348,-3.698525
3,Centro,Justicia (14,40.423957,-3.695747
4,Centro,Universidad (15,40.425264,-3.706606
5,Centro,Sol (16,40.417827,-3.702222
6,Arganzuela,Imperial (21,40.406915,-3.717329
7,Arganzuela,Acacias (22,40.404075,-3.705957
8,Arganzuela,Chopera (23,40.394893,-3.699705
9,Arganzuela,Legazpi (24,40.391172,-3.695190


### 2. Using Forsquare API to get most popular Madrid venues

Documenting credentials for Foursquare API

In [10]:
CLIENT_ID = 'FTDDUPPDRRYADLDFWIYVSI0WLNTMC4Y0RT2ILL1E4EVGKWGD' # your Foursquare ID
CLIENT_SECRET = 'L0FLQBN3OOK1O02NLOGLORIAZ35JBRGMV1CHCGTCXBY4F2NI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FTDDUPPDRRYADLDFWIYVSI0WLNTMC4Y0RT2ILL1E4EVGKWGD
CLIENT_SECRET:L0FLQBN3OOK1O02NLOGLORIAZ35JBRGMV1CHCGTCXBY4F2NI


Creating a function and the GET request URL

In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Using the above function to each neighbourhood and creating a new dataframe called **madrid_venues**

In [12]:
madrid_venues = getNearbyVenues(names=madrid_df['Neighbourhood'],
                                   latitudes=madrid_df['Latitude'],
                                   longitudes=madrid_df['Longitude']
                                  )

Palacio (11
Embajadores (12
Cortes (13
Justicia (14
Universidad (15
Sol (16
Imperial (21
Acacias (22
Chopera (23
Legazpi (24
Delicias (25
Palos de Moguer (26
Atocha (27
Pacífico (31
Adelfas (32
Estrella (33
Ibiza (34
Jerónimos (35
Niño Jesús (36
Recoletos (41
Goya (42
Fuente del Berro (43
Guindalera (44
Lista (45
Castellana (46
El Viso (51
Prosperidad (52
Ciudad Jardín (53
Hispanoamérica (54
Nueva España (55
Castilla (56
Bellas Vistas (61
Cuatro Caminos (62
Castillejos (63
Almenara (64
Valdeacederas (65
Berruguete (66
Gaztambide (71
Arapiles (72
Trafalgar (73
Almagro (74
Ríos Rosas (75
Vallehermoso (76


In [13]:
print(madrid_venues.shape)
madrid_venues.head()

(2560, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Palacio (11,40.415129,-3.715618,Santa Iglesia Catedral de Santa María la Real ...,40.415767,-3.714516,Church
1,Palacio (11,40.415129,-3.715618,Plaza de La Almudena,40.416320,-3.713777,Plaza
2,Palacio (11,40.415129,-3.715618,Cervecería La Mayor,40.415218,-3.712194,Beer Bar
3,Palacio (11,40.415129,-3.715618,Corral de la Morería,40.412619,-3.714249,Performing Arts Venue
4,Palacio (11,40.415129,-3.715618,Taberna Rayuela,40.413179,-3.713496,Tapas Restaurant


Checking how many venues there are for each neighbourhood

In [14]:
madrid_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Acacias (22,52,52,52,52,52,52
Adelfas (32,33,33,33,33,33,33
Almagro (74,100,100,100,100,100,100
Almenara (64,17,17,17,17,17,17
Arapiles (72,72,72,72,72,72,72
Atocha (27,37,37,37,37,37,37
Bellas Vistas (61,25,25,25,25,25,25
Berruguete (66,39,39,39,39,39,39
Castellana (46,100,100,100,100,100,100


Unique categories for venues

In [15]:
print('There are {} uniques categories.'.format(len(madrid_venues['Venue Category'].unique())))

There are 227 uniques categories.


Analyzing neighbourhoods of Madrid, transforming dataframe using dummies method

In [16]:
# one hot encoding
madrid_onehot = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
madrid_onehot['Neighbourhood'] = madrid_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [madrid_onehot.columns[-1]] + list(madrid_onehot.columns[:-1])
madrid_onehot = madrid_onehot[fixed_columns]

madrid_onehot.head()

,Neighbourhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Building,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Photography Lab,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Rental Car Location,Restaurant,Rock Club,Salad Place,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Train Station,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Palacio (11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Palacio (11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [17]:
madrid_onehot.shape

(2560, 228)

Grouping rows by boroughs and by taking the mean of the frequency of occurrence of each category

In [18]:
madrid_grouped = madrid_onehot.groupby('Neighbourhood').mean().reset_index()
madrid_grouped.head()

,Neighbourhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Building,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Photography Lab,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Rental Car Location,Restaurant,Rock Club,Salad Place,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Train Station,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Acacias (22,0.0,0.0,0.000000,0.0,0.0,0.0,0.057692,0.00,0.0,0.000000,0.0,0.0,0.0,0.038462,0.134615,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.019231,0.0,0.0,0.0,0.000000,0.019231,0.00,0.0,0.0,0.019231,0.000000,0.0,0.057692,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.019231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.019231,0.0,0.0,0.000000,0.0,0.019231,0.0,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.019231,0.0,0.0,0.000000,0.019231,0.000000,0.019231,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.019231,0.019231,0.019231,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.019231,0.0,0.019231,0.0,0.

In [19]:
madrid_grouped.shape

(43, 228)

Each neighbourhood with the top 5 most common venues

In [20]:
num_top_venues = 5

for hood in madrid_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = madrid_grouped[madrid_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acacias (22----
                venue  freq
0                 Bar  0.13
1  Spanish Restaurant  0.08
2         Art Gallery  0.06
3                Café  0.06
4              Bakery  0.04


----Adelfas (32----
                  venue  freq
0                   Bar  0.09
1      Tapas Restaurant  0.06
2                Bakery  0.06
3    Spanish Restaurant  0.06
4  Fast Food Restaurant  0.06


----Almagro (74----
                      venue  freq
0        Spanish Restaurant  0.15
1                Restaurant  0.12
2                     Hotel  0.09
3  Mediterranean Restaurant  0.04
4               Supermarket  0.04


----Almenara (64----
                venue  freq
0  Spanish Restaurant  0.24
1         Supermarket  0.12
2               Hotel  0.12
3                Café  0.06
4      Sandwich Place  0.06


----Arapiles (72----
                venue  freq
0  Spanish Restaurant  0.10
1                 Bar  0.08
2              Bakery  0.06
3                Café  0.04
4          Restaurant  0.04




Writing a function to sort the venues in descending order

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating the new dataframe and display the top 10 venues for each borough

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
madrid_venues_sorted = pd.DataFrame(columns=columns)
madrid_venues_sorted['Neighbourhood'] = madrid_grouped['Neighbourhood']

for ind in np.arange(madrid_grouped.shape[0]):
    madrid_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

madrid_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acacias (22,Bar,Spanish Restaurant,Art Gallery,Café,Plaza,Restaurant,Bakery,Tapas Restaurant,Park,Pizza Place
1,Adelfas (32,Bar,Tapas Restaurant,Hotel,Bakery,Spanish Restaurant,Fast Food Restaurant,Soccer Field,Café,Gift Shop,Bookstore
2,Almagro (74,Spanish Restaurant,Restaurant,Hotel,Supermarket,Mediterranean Restaurant,Bar,Nightclub,French Restaurant,Salad Place,Asian Restaurant
3,Almenara (64,Spanish Restaurant,Supermarket,Hotel,Chinese Restaurant,Metro Station,Gym / Fitness Center,Bar,Library,Frozen Yogurt Shop,Café
4,Arapiles (72,Spanish Restaurant,Bar,Bakery,Café,Hotel,Restaurant,Sandwich Place,Multiplex,Coffee Shop,Sports Bar


### 3. Clustering

Runing k-means to cluster the boroughs into X clusters

In [41]:
# set number of clusters
kclusters = 6

madrid_grouped_clustering = madrid_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 1, 2, 0, 4, 2, 2, 1, 1], dtype=int32)

Creating a new dataframe that includes the cluster as well as the top 10 venues for each borough

In [42]:
# adding clustering labels
madrid_venues_sorted.insert(0, 'Cluster labels', kmeans.labels_)

madrid_merged = madrid_df

# merging toronto_grouped with toronto_data to add latitude/longitude for each borough
madrid_merged = madrid_merged.join(madrid_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

madrid_merged.head()

,District,Neighbourhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centro,Palacio (11,40.415129,-3.715618,0,Spanish Restaurant,Plaza,Tapas Restaurant,Restaurant,Bar,Cocktail Bar,Historic Site,Vegetarian / Vegan Restaurant,Mexican Restaurant,Music Venue
1,Centro,Embajadores (12,40.409681,-3.701644,4,Bar,Tapas Restaurant,Café,Spanish Restaurant,Hostel,Pizza Place,Theater,Hotel,Coffee Shop,Vegetarian / Vegan Restaurant
2,Centro,Cortes (13,40.414348,-3.698525,3,Spanish Restaurant,Hotel,Restaurant,Theater,Plaza,Bar,Tapas Restaurant,Jazz Club,Mediterranean Restaurant,Cocktail Bar
3,Centro,Justicia (14,40.423957,-3.695747,0,Bakery,Italian Restaurant,Hotel,Spanish Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Cocktail Bar,Flower Shop,Gay Bar,Bar
4,Centro,Universidad (15,40.425264,-3.706606,4,Bar,Tapas Restaurant,Cocktail Bar,Café,Theater,Spanish Restaurant,Breakfast Spot,Restaurant,Argentinian Restaurant,Plaza


Getting Madrid latitude and longitude

In [25]:
address = 'Madrid'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid are 40.4167047, -3.7035825.


Visualizing clusters

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(madrid_merged['Latitude'], madrid_merged['Longitude'], madrid_merged['Neighbourhood'], madrid_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Examining clusters**

The largest cluster.
Cluster 0: Reataurants (mostly Spanish restaurants), bars, parks, few coffee shops and historic sites

In [44]:
madrid_merged.loc[madrid_merged['Cluster labels'] == 0, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Palacio (11,Spanish Restaurant,Plaza,Tapas Restaurant,Restaurant,Bar,Cocktail Bar,Historic Site,Vegetarian / Vegan Restaurant,Mexican Restaurant,Music Venue
3,Justicia (14,Bakery,Italian Restaurant,Hotel,Spanish Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Cocktail Bar,Flower Shop,Gay Bar,Bar
8,Chopera (23,Bar,Park,Art Gallery,Tapas Restaurant,Coffee Shop,Burger Joint,Pet Store,Café,Skate Park,Shopping Mall
9,Legazpi (24,Plaza,Bar,Spanish Restaurant,Tapas Restaurant,Chinese Restaurant,Art Gallery,BBQ Joint,Burger Joint,Beer Garden,Argentinian Restaurant
10,Delicias (25,Restaurant,Bakery,Coffee Shop,Ice Cream Shop,Gym / Fitness Center,Breakfast Spot,Flea Market,Snack Place,Museum,Mediterranean Restaurant
11,Palos de Moguer (26,Spanish Restaurant,Restaurant,Tapas Restaurant,Beer Garden,Sandwich Place,Grocery Store,Brewery,Plaza,Garden,Breakfast Spot
15,Estrella (33,Park,Spanish Restaurant,Asian Restaurant,Bar,Restaurant,Music Venue,Chinese Restaurant,Metro Station,Big Box Store,Pool
18,Niño Jesús (36,Park,Spanish Restaurant,Bar,Café,Asian Restaurant,Italian Restaurant,Dessert Shop,Cocktail Bar,Metro Station,Mexican Restaurant
21,Fuente del Berro (43,Spanish Restaurant,Gym / Fitness Center,Tapas Restaurant,Bar,Park,Brewery,Supermarket,Grocery Store,Burger Joint,Sporting Goods Shop
25,El Viso (51,Spanish Restaurant,Mediterranean Restaurant,Accessories Store,Theme Restaurant,Pizza Place,Plaza,Restaurant,Flea Market,Burger Joint,Gourmet Shop


Cluster 1: A lot of restaurants (mostly Spanish), hotels and some stores.

In [45]:
madrid_merged.loc[madrid_merged['Cluster labels'] == 1, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Ibiza (34,Spanish Restaurant,Tapas Restaurant,Restaurant,Bakery,Mediterranean Restaurant,Breakfast Spot,Seafood Restaurant,Asian Restaurant,Gastropub,Italian Restaurant
19,Recoletos (41,Restaurant,Spanish Restaurant,Italian Restaurant,Clothing Store,Furniture / Home Store,Japanese Restaurant,Tapas Restaurant,Coffee Shop,Accessories Store,Hotel
20,Goya (42,Spanish Restaurant,Restaurant,Tapas Restaurant,Seafood Restaurant,Burger Joint,Café,Supermarket,Bar,Chinese Restaurant,Snack Place
23,Lista (45,Spanish Restaurant,Restaurant,Seafood Restaurant,Hotel,Japanese Restaurant,Tapas Restaurant,Burger Joint,Mediterranean Restaurant,Mexican Restaurant,Gym
24,Castellana (46,Spanish Restaurant,Restaurant,Coffee Shop,Boutique,Tapas Restaurant,Hotel,Gastropub,Japanese Restaurant,Bistro,Jewelry Store
30,Castilla (56,Spanish Restaurant,Hotel,Restaurant,Seafood Restaurant,Indian Restaurant,Tapas Restaurant,Coffee Shop,Supermarket,Bar,Bakery
35,Valdeacederas (65,Spanish Restaurant,Restaurant,Coffee Shop,Grocery Store,Chinese Restaurant,Hotel,Italian Restaurant,Nightclub,Bakery,Burger Joint
40,Almagro (74,Spanish Restaurant,Restaurant,Hotel,Supermarket,Mediterranean Restaurant,Bar,Nightclub,French Restaurant,Salad Place,Asian Restaurant


Cluster 2: Restaurants and a lot of bars.

In [46]:
madrid_merged.loc[madrid_merged['Cluster labels'] == 2, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Prosperidad (52,Spanish Restaurant,Bar,Restaurant,Pizza Place,Supermarket,Café,Tapas Restaurant,Bakery,Grocery Store,Japanese Restaurant
27,Ciudad Jardín (53,Spanish Restaurant,Seafood Restaurant,Plaza,Bar,Grocery Store,Gym,Restaurant,Health & Beauty Service,Supermarket,Gastropub
28,Hispanoamérica (54,Spanish Restaurant,Bar,Restaurant,Café,Grocery Store,Pub,Bakery,Diner,Gym,Plaza
31,Bellas Vistas (61,Spanish Restaurant,Bar,Pub,Seafood Restaurant,Pizza Place,Donut Shop,Steakhouse,Supermarket,Tapas Restaurant,Gym
33,Castillejos (63,Bar,Spanish Restaurant,Supermarket,Pizza Place,Electronics Store,Coffee Shop,Seafood Restaurant,Kebab Restaurant,Snack Place,Café
34,Almenara (64,Spanish Restaurant,Supermarket,Hotel,Chinese Restaurant,Metro Station,Gym / Fitness Center,Bar,Library,Frozen Yogurt Shop,Café
36,Berruguete (66,Supermarket,Bar,Spanish Restaurant,Kebab Restaurant,Bakery,Pizza Place,Breakfast Spot,Music Venue,Coffee Shop,Metro Station


Cluster 3: A lot of hotels, restaurants, some theaters and plazas. 

In [47]:
madrid_merged.loc[madrid_merged['Cluster labels'] == 3, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Cortes (13,Spanish Restaurant,Hotel,Restaurant,Theater,Plaza,Bar,Tapas Restaurant,Jazz Club,Mediterranean Restaurant,Cocktail Bar
5,Sol (16,Hotel,Spanish Restaurant,Tapas Restaurant,Restaurant,Clothing Store,Cocktail Bar,Plaza,Art Museum,Gourmet Shop,Theater
6,Imperial (21,Hotel,Spanish Restaurant,Coffee Shop,Garden,Korean Restaurant,Gym,Gym / Fitness Center,Mediterranean Restaurant,Beer Garden,Park
22,Guindalera (44,Hotel,Indian Restaurant,Seafood Restaurant,Restaurant,Spanish Restaurant,Theater,Post Office,Plaza,Supermarket,Grocery Store


Cluster 4: Mostly bars and coffee shops, bakeries and Spanish/Tapas restaurants.

In [48]:
madrid_merged.loc[madrid_merged['Cluster labels'] == 4, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Embajadores (12,Bar,Tapas Restaurant,Café,Spanish Restaurant,Hostel,Pizza Place,Theater,Hotel,Coffee Shop,Vegetarian / Vegan Restaurant
4,Universidad (15,Bar,Tapas Restaurant,Cocktail Bar,Café,Theater,Spanish Restaurant,Breakfast Spot,Restaurant,Argentinian Restaurant,Plaza
7,Acacias (22,Bar,Spanish Restaurant,Art Gallery,Café,Plaza,Restaurant,Bakery,Tapas Restaurant,Park,Pizza Place
12,Atocha (27,Restaurant,Spanish Restaurant,Bar,Café,Bakery,Italian Restaurant,Museum,Breakfast Spot,Polish Restaurant,Plaza
13,Pacífico (31,Bar,Spanish Restaurant,Bakery,Café,Restaurant,Museum,Pizza Place,Asian Restaurant,Art Gallery,Italian Restaurant
14,Adelfas (32,Bar,Tapas Restaurant,Hotel,Bakery,Spanish Restaurant,Fast Food Restaurant,Soccer Field,Café,Gift Shop,Bookstore
29,Nueva España (55,Restaurant,Bar,Metro Station,Tapas Restaurant,Steakhouse,Gym,Bakery,Big Box Store,Peruvian Restaurant,Plaza
42,Vallehermoso (76,Bar,Restaurant,Spanish Restaurant,Sandwich Place,Pizza Place,Tapas Restaurant,Nightclub,Bakery,Burger Joint,Food & Drink Shop


Cluster 5: "Green" cluster, only one object with park.

In [49]:
madrid_merged.loc[madrid_merged['Cluster labels'] == 5, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Jerónimos (35,Park,Snack Place,Diner,Plaza,Coffee Shop,Art Museum,Café,Fountain,Monument / Landmark,Mediterranean Restaurant


# BARCELONA

### 1. Creating dataframe for Barcelona

Reading Wikipedia file of Barcelonas' districts and neighbourhoods to dataframe

In [54]:
barcelona_df=pd.read_html("https://es.wikipedia.org/wiki/Distritos_de_Barcelona")[0]
barcelona_df.head()

,Nº,Distrito,Imagen,Superficie km²,Población (2016)[1]​,Densidad hab/km²,Barrios (nº),Regidor
0,1,Ciutat Vella,NaN,449,100 070,"22 424,28","El Raval (1), Barrio Gótico (2), La Barcelonet...",Gala Pin Ferrando (Barcelona en Comú)
1,2,Eixample,NaN,746,264 305,"35 330,43","El Fort Pienc (5), Sagrada Familia (6), Dreta ...",Agustí Colom Cabau (Barcelona en Comú)
2,3,Sants - Montjuïc,NaN,2135,180 977,846951,"Pueblo Seco (11), La Marina del Prat Vermell (...",Jaume Asens Llodrà (Barcelona en Comú)
3,4,Les Corts,NaN,608,81 642,"13 355,26","Les Corts (19), La Maternidad y San Ramón (20)...",Laura Pérez Castaño (Barcelona en Comú)
4,5,Sarrià - Sant Gervasi,NaN,2009,148 026,725540,"Vallvidrera, Tibidabo i les Planes (22), Sarri...",Gerardo Pisarello Prados (Barcelona en Comú)


Dropping unnecessary columns and renaming columns I will use for further analysis

In [55]:
barcelona_df.drop(['Nº', 'Imagen', 'Superficie km²', 'Población (2016)[1]​', 'Densidad hab/km²', 'Regidor'], axis=1, inplace=True)
barcelona_df.rename(columns={'Distrito': 'District', 'Barrios (nº)': 'Neighbourhood'}, inplace=True)
barcelona_df

,District,Neighbourhood
0,Ciutat Vella,"El Raval (1), Barrio Gótico (2), La Barcelonet..."
1,Eixample,"El Fort Pienc (5), Sagrada Familia (6), Dreta ..."
2,Sants - Montjuïc,"Pueblo Seco (11), La Marina del Prat Vermell (..."
3,Les Corts,"Les Corts (19), La Maternidad y San Ramón (20)..."
4,Sarrià - Sant Gervasi,"Vallvidrera, Tibidabo i les Planes (22), Sarri..."
5,Gràcia,"Vallcarca y los Penitentes (28), El Coll (29),..."
6,Horta - Guinardó,"Baix Guinardó (33), Can Baró (34), El Guinardó..."
7,Nou Barris,"Vilapicina y La Torre Llobeta (44), Porta (45)..."
8,Sant Andreu,"La Trinitat Vella (57), Baró de Viver (58), El..."
9,Sant Martí,"El Campo del Arpa del Clot (64), El Clot (65),..."


Selecting districts for further analysis

In [63]:
barcelona_df=barcelona_df.iloc[[0, 1, 2, 9]]
barcelona_df

,District,Neighbourhood
0,Ciutat Vella,"El Raval (1), Barrio Gótico (2), La Barcelonet..."
1,Eixample,"El Fort Pienc (5), Sagrada Familia (6), Dreta ..."
2,Sants - Montjuïc,"Pueblo Seco (11), La Marina del Prat Vermell (..."
9,Sant Martí,"El Campo del Arpa del Clot (64), El Clot (65),..."


Exploding dataframe to create a new row for each neighbourhood

In [64]:
barcelona_df=barcelona_df.assign(Neighbourhood=barcelona_df.Neighbourhood.str.split(',')).explode('Neighbourhood').reset_index(drop=True)
barcelona_df

,District,Neighbourhood
0,Ciutat Vella,El Raval (1)
1,Ciutat Vella,Barrio Gótico (2)
2,Ciutat Vella,La Barceloneta (3) y Sant Pere
3,Ciutat Vella,Santa Caterina i la Ribera (4)
4,Eixample,El Fort Pienc (5)
5,Eixample,Sagrada Familia (6)
6,Eixample,Dreta de l'Eixample (7)
7,Eixample,La Antigua Izquierda del Ensanche (8)
8,Eixample,La Nueva Izquierda del Ensanche (9) y Sant An...
9,Sants - Montjuïc,Pueblo Seco (11)


In [65]:
barcelona_df.shape

(27, 2)

Getting latitude and longitude for each neighbourhood

In [126]:
address = 'Barcelona, El Raval'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, El Raval are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, El Raval are 41.3795176, 2.1683678.


In [127]:
address = 'Barcelona, Barrio Gótico'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Barrio Gótico are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Barrio Gótico are 41.3833947, 2.1769119.


In [128]:
address = 'Barcelona, La Barceloneta'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, La Barceloneta are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, La Barceloneta are 41.3806533, 2.1899274.


In [129]:
address = 'Barcelona, Santa Caterina i la Ribera'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Santa Caterina i la Ribera are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Santa Caterina i la Ribera are 41.3866873, 2.18414740213414.


In [66]:
address = 'Barcelona, El Fort Pienc'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, El Fort Pienc {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, El Fort Pienc 41.3959246, 2.1823245.


In [67]:
address = 'Barcelona, Sagrada Familia'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Sagrada Familia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Sagrada Familia 41.4034789, 2.17441033300971.


In [70]:
address = 'Barcelona, Dreta de l Eixample' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Dreta de l Eixample {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Dreta de l Eixample 41.398694, 2.1782798.


In [73]:
address = 'Barcelona, La Antigua' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, La Antigua Izquierda del Ensanche {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, La Antigua Izquierda del Ensanche 41.3980086, 2.1492952.


In [81]:
address = 'Barcelona, La Nova Esquerra de l Eixample' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, La Nueva Izquierda del Ensanche {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, La Nueva Izquierda del Ensanche 41.3878717, 2.1454459.


In [86]:
address = 'Barcelona, Sants - Montjuïc, El Poble-sec' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Pueblo Seco {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Pueblo Seco 41.3728061, 2.1619718.


In [87]:
address = 'Barcelona, Sants - Montjuïc, La Marina del Prat Vermell ' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, La Marina del Prat Vermell  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, La Marina del Prat Vermell  41.3538419, 2.1423184.


In [88]:
address = 'Barcelona, Sants - Montjuïc, La Marina de Port  ' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, La Marina de Port {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, La Marina de Port 41.3602964, 2.1375842.


In [89]:
address = 'Barcelona, Sants - Montjuïc, La Font de la Guatlla ' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, La Font de la Guatlla {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, La Font de la Guatlla 41.3707824, 2.1446756.


In [90]:
address = 'Barcelona, Sants - Montjuïc, Hostafrancs' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Hostafrancs {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Hostafrancs 41.3750877, 2.1429334.


In [91]:
address = 'Barcelona, Sants - Montjuïc, La Bordeta ' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, La Bordeta  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, La Bordeta  41.3691417, 2.1373733.


In [92]:
address = 'Barcelona, Sants - Montjuïc, Sants-Badal  ' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Sants-Badal  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Sants-Badal  41.375466, 2.1277096.


In [93]:
address = 'Barcelona, Sants - Montjuïc, Sants' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Sants {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Sants  41.3753288, 2.1349117.


In [94]:
address = 'Barcelona, Sants - Montjuïc, Zona Franca  ' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Zona Franca  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Zona Franca  41.3334396, 2.1298587.


In [102]:
address = 'Barcelona, Sant Martí, El Camp' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, El Campo del Arpa del Clot  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, El Campo del Arpa del Clot  41.4067585, 2.20368795208359.


In [103]:
address = 'Barcelona, Sant Martí, El Clot ' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, El Clot  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, El Clot  41.4099525, 2.1908234.


In [104]:
address = 'Barcelona, Sant Martí, El Parc i la Llacuna del Poblenou ' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, El Parc i la Llacuna del Poblenou  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, El Parc i la Llacuna del Poblenou  41.4007334, 2.191342.


In [105]:
address = 'Barcelona, Sant Martí, La Vila Olímpica del Poblenou ' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, La Vila Olímpica del Poblenou {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, La Vila Olímpica del Poblenou 41.38980905, 2.19555531435746.


In [107]:
address = 'Barcelona, Sant Martí, El Poblenou' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Pueblo Nuevo  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Pueblo Nuevo  41.400527, 2.2017292.


In [108]:
address = 'Barcelona, Sant Martí, Diagonal Mar i Front Marítim del Poblenou' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Diagonal Mar i Front Marítim del Poblenou  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Diagonal Mar i Front Marítim del Poblenou  41.4032638, 2.21348059911793.


In [109]:
address = 'Barcelona, Sant Martí, El Besós y el Maresme' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, El Besós y el Maresme {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, El Besós y el Maresme 41.4104628, 2.2156503.


In [110]:
address = 'Barcelona, Sant Martí, Provençals del Poblenou ' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Provençals del Poblenou  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Provençals del Poblenou  41.41236025, 2.20488533418194.


In [112]:
address = 'Barcelona, Sant Martí, Sant Martí de Provençals' 

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, San Martín de Provensals {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, San Martín de Provensals 41.4165186, 2.1989683.


Adding columns with latitude and longitude to Barcelona dataframe

In [160]:
b_latitude = [41.3795176, 41.3833947, 41.3806533, 41.3866873, 41.3959246, 41.4034789, 41.398694, 41.388923, 41.383052, 41.3728061, 41.3538419,
             41.3602964, 41.3707824, 41.3750877, 41.3691417, 41.375466, 41.3753288, 41.3334396, 41.411999, 41.4099525, 41.4007334, 41.38980905,
             41.400527, 41.4032638, 41.4104628, 41.41236025, 41.4165186]
b_longitude = [2.1683678, 2.1769119, 2.1899274, 2.18414740213414, 2.1823245, 2.17441033300971, 2.1782798, 2.156196, 2.149984, 2.1619718, 2.1423184,
              2.1375842, 2.1446756, 2.1429334, 2.1373733, 2.1277096, 2.1349117, 2.1298587, 2.182910, 2.1908234, 2.191342, 2.19555531435746,
              2.2017292, 2.21348059911793, 2.2156503, 2.20488533418194, 2.1989683]
barcelona_df['Latitude'] = b_latitude 
barcelona_df['Longitude'] = b_longitude
barcelona_df

,District,Neighbourhood,Latitude,Longitude
0,Ciutat Vella,El Raval (1),41.379518,2.168368
1,Ciutat Vella,Barrio Gótico (2),41.383395,2.176912
2,Ciutat Vella,La Barceloneta (3) y Sant Pere,41.380653,2.189927
3,Ciutat Vella,Santa Caterina i la Ribera (4),41.386687,2.184147
4,Eixample,El Fort Pienc (5),41.395925,2.182325
5,Eixample,Sagrada Familia (6),41.403479,2.174410
6,Eixample,Dreta de l'Eixample (7),41.398694,2.178280
7,Eixample,La Antigua Izquierda del Ensanche (8),41.388923,2.156196
8,Eixample,La Nueva Izquierda del Ensanche (9) y Sant An...,41.383052,2.149984
9,Sants - Montjuïc,Pueblo Seco (11),41.372806,2.161972


Dropping one neighbourhood, as it has no venues as I learned later :)

In [161]:
barcelona_df = barcelona_df.drop([17])
barcelona_df = barcelona_df.reset_index(drop=True)
barcelona_df

,District,Neighbourhood,Latitude,Longitude
0,Ciutat Vella,El Raval (1),41.379518,2.168368
1,Ciutat Vella,Barrio Gótico (2),41.383395,2.176912
2,Ciutat Vella,La Barceloneta (3) y Sant Pere,41.380653,2.189927
3,Ciutat Vella,Santa Caterina i la Ribera (4),41.386687,2.184147
4,Eixample,El Fort Pienc (5),41.395925,2.182325
5,Eixample,Sagrada Familia (6),41.403479,2.174410
6,Eixample,Dreta de l'Eixample (7),41.398694,2.178280
7,Eixample,La Antigua Izquierda del Ensanche (8),41.388923,2.156196
8,Eixample,La Nueva Izquierda del Ensanche (9) y Sant An...,41.383052,2.149984
9,Sants - Montjuïc,Pueblo Seco (11),41.372806,2.161972


### 2. Using Forsquare API to get most popular Barcelona venues

Creating a function and the GET request URL

In [141]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

def getNearbyVenues1(names, latitudes, longitudes, radius=500):
    
    venues_list1=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # API request URL
        url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # GET request
        results2 = requests.get(url2).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list1.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results2])

    nearby_venues1 = pd.DataFrame([item for venue_list1 in venues_list1 for item in venue_list1])
    nearby_venues1.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues1)

Using the above function to each neighbourhood and creating a new dataframe called **barcelona_venues**

In [142]:
barcelona_venues = getNearbyVenues1(names=barcelona_df['Neighbourhood'],
                                   latitudes=barcelona_df['Latitude'],
                                   longitudes=barcelona_df['Longitude']
                                  )

El Raval (1)
 Barrio Gótico (2)
 La Barceloneta (3) y Sant Pere
 Santa Caterina i la Ribera (4)
El Fort Pienc (5)
 Sagrada Familia (6)
 Dreta de l'Eixample (7)
 La Antigua Izquierda del Ensanche (8)
 La Nueva Izquierda del Ensanche (9) y Sant Antoni (10)
Pueblo Seco (11)
 La Marina del Prat Vermell (12)
 La Marina de Port (13)
 La Font de la Guatlla (14)
 Hostafrancs (15)
 La Bordeta (16)
 Sants-Badal (17)
 Sants (18)
 Zona Franca (*) y Montjuic (*)
El Campo del Arpa del Clot (64)
 El Clot (65)
 El Parc i la Llacuna del Poblenou (66)
 La Vila Olímpica del Poblenou (67)
 Pueblo Nuevo (68)
 Diagonal Mar i Front Marítim del Poblenou (69)
 El Besós y el Maresme (70)
 Provençals del Poblenou (71)
 San Martín de Provensals (72) y La Verneda y la Paz (73)


In [146]:
print(barcelona_venues.shape)
barcelona_venues.head()

(1779, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,El Raval (1),41.379518,2.168368,Cera 23,41.378947,2.166180,Spanish Restaurant
1,El Raval (1),41.379518,2.168368,Chulapio,41.379264,2.165905,Cocktail Bar
2,El Raval (1),41.379518,2.168368,Arume,41.378953,2.166008,Spanish Restaurant
3,El Raval (1),41.379518,2.168368,A Tu Bola,41.380096,2.169054,Tapas Restaurant
4,El Raval (1),41.379518,2.168368,La Monroe,41.378795,2.170692,Spanish Restaurant


Checking how many venues there are for each neighbourhood

In [147]:
barcelona_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Barrio Gótico (2),100,100,100,100,100,100
Diagonal Mar i Front Marítim del Poblenou (69),30,30,30,30,30,30
Dreta de l'Eixample (7),69,69,69,69,69,69
El Besós y el Maresme (70),85,85,85,85,85,85
El Clot (65),78,78,78,78,78,78
El Parc i la Llacuna del Poblenou (66),95,95,95,95,95,95
Hostafrancs (15),62,62,62,62,62,62
La Antigua Izquierda del Ensanche (8),100,100,100,100,100,100
La Barceloneta (3) y Sant Pere,100,100,100,100,100,100


Unique categories for venues

In [148]:
print('There are {} uniques categories.'.format(len(barcelona_venues['Venue Category'].unique())))

There are 235 uniques categories.


Analyzing neighbourhoods of Barcelona, transforming dataframe using dummies method

In [149]:
# one hot encoding
barcelona_onehot = pd.get_dummies(barcelona_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
barcelona_onehot['Neighbourhood'] = barcelona_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [barcelona_onehot.columns[-1]] + list(barcelona_onehot.columns[:-1])
barcelona_onehot = barcelona_onehot[fixed_columns]

barcelona_onehot.head()

,Neighbourhood,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Building,Burger Joint,Bus Station,Cable Car,Cafeteria,Café,Camera Store,Candy Store,Car Wash,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paella Restaurant,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Train Station,Tram Station,Transportation Service,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,El Raval (1),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,El Raval (1),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [150]:
barcelona_onehot.shape

(1779, 236)

Grouping rows by boroughs and by taking the mean of the frequency of occurrence of each category

In [151]:
barcelona_grouped = barcelona_onehot.groupby('Neighbourhood').mean().reset_index()
barcelona_grouped.head()

,Neighbourhood,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Building,Burger Joint,Bus Station,Cable Car,Cafeteria,Café,Camera Store,Candy Store,Car Wash,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paella Restaurant,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Train Station,Tram Station,Transportation Service,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Barrio Gótico (2),0.000000,0.0,0.0,0.000000,0.000000,0.01,0.01,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.01,0.0,0.010000,0.040000,0.000000,0.000000,0.0,0.010000,0.0,0.000000,0.000000,0.010000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.01,0.000000,0.0,0.020000,0.0,0.0,0.0,0.040000,0.01,0.01,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.050000,0.030000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.03,0.0,0.010000,0.0,0.000000,0.0,0.000000,0.01,0.0,0.0,0.01,0.010000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.010000,0.01,0.000000,0.0,0.000000,0.0,0.000000,0.010000,0.010000,0.0,0.0,0.000000,0.000000,0.

In [152]:
barcelona_grouped.shape

(26, 236)

Each neighbourhood with the top 5 most common venues

In [153]:
num_top_venues = 5

for hood in barcelona_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = barcelona_grouped[barcelona_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Barrio Gótico (2)----
                venue  freq
0               Plaza  0.09
1    Tapas Restaurant  0.07
2               Hotel  0.06
3        Cocktail Bar  0.05
4  Spanish Restaurant  0.05


---- Diagonal Mar i Front Marítim del Poblenou (69)----
                      venue  freq
0  Mediterranean Restaurant  0.13
1                 Beach Bar  0.13
2                Restaurant  0.13
3        Athletics & Sports  0.07
4                     Beach  0.07


---- Dreta de l'Eixample (7)----
                venue  freq
0          Restaurant  0.07
1  Mexican Restaurant  0.07
2    Tapas Restaurant  0.06
3               Hotel  0.06
4         Pizza Place  0.04


---- El Besós y el Maresme (70)----
                venue  freq
0      Clothing Store  0.07
1  Italian Restaurant  0.07
2               Hotel  0.07
3                Café  0.06
4        Burger Joint  0.04


---- El Clot (65)----
                      venue  freq
0        Spanish Restaurant  0.09
1  Mediterranean Restaurant  0.08
2       

Writing a function to sort the venues in descending order

In [162]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating the new dataframe and display the top 10 venues for each borough

In [199]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
barcelona_venues_sorted = pd.DataFrame(columns=columns)
barcelona_venues_sorted['Neighbourhood'] = barcelona_grouped['Neighbourhood']

for ind in np.arange(barcelona_grouped.shape[0]):
    barcelona_venues_sorted.iloc[ind, 1:] = return_most_common_venues(barcelona_grouped.iloc[ind, :], num_top_venues)

barcelona_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barrio Gótico (2),Plaza,Tapas Restaurant,Hotel,Spanish Restaurant,Cocktail Bar,Bar,Café,Pizza Place,Ice Cream Shop,Coffee Shop
1,Diagonal Mar i Front Marítim del Poblenou (69),Restaurant,Beach Bar,Mediterranean Restaurant,Beach,Café,Athletics & Sports,Coffee Shop,Thai Restaurant,Park,Cocktail Bar
2,Dreta de l'Eixample (7),Mexican Restaurant,Restaurant,Hotel,Tapas Restaurant,Café,Coffee Shop,Spanish Restaurant,Burger Joint,Pizza Place,Hostel
3,El Besós y el Maresme (70),Italian Restaurant,Clothing Store,Hotel,Café,Coffee Shop,Restaurant,Tapas Restaurant,Burger Joint,Sandwich Place,Ice Cream Shop
4,El Clot (65),Spanish Restaurant,Mediterranean Restaurant,Bakery,Tapas Restaurant,Café,Supermarket,Hotel,Restaurant,Park,Coffee Shop


### 3. Clustering

Runing k-means to cluster the boroughs into 5 clusters

In [200]:
# set number of clusters
kclusters = 5

barcelona_grouped_clustering = barcelona_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(barcelona_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 2, 1, 1, 1, 1, 1, 2], dtype=int32)

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood

In [201]:
# adding clustering labels
barcelona_venues_sorted.insert(0, 'Cluster labels', kmeans.labels_)

barcelona_merged = barcelona_df

# merging toronto_grouped with toronto_data to add latitude/longitude for each borough
barcelona_merged = barcelona_merged.join(barcelona_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

barcelona_merged.head()

,District,Neighbourhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ciutat Vella,El Raval (1),41.379518,2.168368,1,Bar,Spanish Restaurant,Cocktail Bar,Tapas Restaurant,Pizza Place,Mediterranean Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Theater,Japanese Restaurant
1,Ciutat Vella,Barrio Gótico (2),41.383395,2.176912,1,Plaza,Tapas Restaurant,Hotel,Spanish Restaurant,Cocktail Bar,Bar,Café,Pizza Place,Ice Cream Shop,Coffee Shop
2,Ciutat Vella,La Barceloneta (3) y Sant Pere,41.380653,2.189927,1,Tapas Restaurant,Bar,Mediterranean Restaurant,Paella Restaurant,Spanish Restaurant,Seafood Restaurant,Pizza Place,Burger Joint,Ice Cream Shop,Restaurant
3,Ciutat Vella,Santa Caterina i la Ribera (4),41.386687,2.184147,1,Tapas Restaurant,Bar,Cocktail Bar,Wine Bar,Restaurant,Café,Hotel,Mexican Restaurant,Italian Restaurant,Mediterranean Restaurant
4,Eixample,El Fort Pienc (5),41.395925,2.182325,2,Coffee Shop,Chinese Restaurant,Spanish Restaurant,Mediterranean Restaurant,Mexican Restaurant,Restaurant,Yoga Studio,Concert Hall,Stadium,Soccer Stadium


Getting Barcelona latitude and longitude

In [166]:
address = 'Barcelona'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona are 41.3828939, 2.1774322.


Visualizing clusters

In [209]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(barcelona_merged['Latitude'], barcelona_merged['Longitude'], barcelona_merged['Neighbourhood'], barcelona_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Examining clusters**

Cluster 0: Has only one object, "Beach" cluster

In [203]:
barcelona_merged.loc[barcelona_merged['Cluster labels'] == 0, barcelona_merged.columns[[1] + list(range(5, barcelona_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Diagonal Mar i Front Marítim del Poblenou (69),Restaurant,Beach Bar,Mediterranean Restaurant,Beach,Café,Athletics & Sports,Coffee Shop,Thai Restaurant,Park,Cocktail Bar


Cluster 1: Restaurants, bars and coffee shops

In [204]:
barcelona_merged.loc[barcelona_merged['Cluster labels'] == 1, barcelona_merged.columns[[1] + list(range(5, barcelona_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,El Raval (1),Bar,Spanish Restaurant,Cocktail Bar,Tapas Restaurant,Pizza Place,Mediterranean Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Theater,Japanese Restaurant
1,Barrio Gótico (2),Plaza,Tapas Restaurant,Hotel,Spanish Restaurant,Cocktail Bar,Bar,Café,Pizza Place,Ice Cream Shop,Coffee Shop
2,La Barceloneta (3) y Sant Pere,Tapas Restaurant,Bar,Mediterranean Restaurant,Paella Restaurant,Spanish Restaurant,Seafood Restaurant,Pizza Place,Burger Joint,Ice Cream Shop,Restaurant
3,Santa Caterina i la Ribera (4),Tapas Restaurant,Bar,Cocktail Bar,Wine Bar,Restaurant,Café,Hotel,Mexican Restaurant,Italian Restaurant,Mediterranean Restaurant
7,La Antigua Izquierda del Ensanche (8),Hotel,Spanish Restaurant,Mediterranean Restaurant,Cocktail Bar,Japanese Restaurant,Tapas Restaurant,Beer Bar,Sandwich Place,Restaurant,Bakery
8,La Nueva Izquierda del Ensanche (9) y Sant An...,Mediterranean Restaurant,Café,Spanish Restaurant,Hotel,Tapas Restaurant,Brewery,Supermarket,Bar,Nightclub,Indian Restaurant
9,Pueblo Seco (11),Tapas Restaurant,Mediterranean Restaurant,Café,Bar,Italian Restaurant,Theater,Hotel,Pizza Place,Cocktail Bar,Spanish Restaurant
12,La Font de la Guatlla (14),Spanish Restaurant,Plaza,Supermarket,Tapas Restaurant,Pizza Place,Gay Bar,Restaurant,Bookstore,Breakfast Spot,Bakery
13,Hostafrancs (15),Pizza Place,Tapas Restaurant,Mediterranean Restaurant,Hotel,Spanish Restaurant,Brewery,Hot Dog Joint,Nightclub,Bar,Gym / Fitness Center
16,Sants (18),Tapas Restaurant,Mediterranean Restaurant,Bar,Café,Italian Restaurant,Gym / Fitness Center,Wine Bar,Middle Eastern Restaurant,Hostel,Brewery


Cluster 2: Coffee shops, restaurants, hotels

In [205]:
barcelona_merged.loc[barcelona_merged['Cluster labels'] == 2, barcelona_merged.columns[[1] + list(range(5, barcelona_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,El Fort Pienc (5),Coffee Shop,Chinese Restaurant,Spanish Restaurant,Mediterranean Restaurant,Mexican Restaurant,Restaurant,Yoga Studio,Concert Hall,Stadium,Soccer Stadium
5,Sagrada Familia (6),Restaurant,Italian Restaurant,Plaza,Burger Joint,Hotel,Café,Bakery,Pizza Place,Mexican Restaurant,Wine Bar
6,Dreta de l'Eixample (7),Mexican Restaurant,Restaurant,Hotel,Tapas Restaurant,Café,Coffee Shop,Spanish Restaurant,Burger Joint,Pizza Place,Hostel
14,La Bordeta (16),Pizza Place,Grocery Store,Supermarket,Restaurant,Burger Joint,Hotel,Tapas Restaurant,Theater,Falafel Restaurant,Café
15,Sants-Badal (17),Bakery,Pizza Place,Café,Tapas Restaurant,Supermarket,Bar,Hostel,Farmers Market,Hotel,Mexican Restaurant
17,El Campo del Arpa del Clot (64),Bakery,Spanish Restaurant,Restaurant,Café,Italian Restaurant,Pizza Place,Hotel,Chinese Restaurant,Cocktail Bar,Burger Joint
20,La Vila Olímpica del Poblenou (67),Café,Spanish Restaurant,Nightclub,Italian Restaurant,Restaurant,Mediterranean Restaurant,Park,Hotel,Lounge,Hookah Bar
21,Pueblo Nuevo (68),Mediterranean Restaurant,Restaurant,Spanish Restaurant,Bakery,Italian Restaurant,Tapas Restaurant,Indian Restaurant,Pizza Place,Café,Gastropub
23,El Besós y el Maresme (70),Italian Restaurant,Clothing Store,Hotel,Café,Coffee Shop,Restaurant,Tapas Restaurant,Burger Joint,Sandwich Place,Ice Cream Shop


Cluster 3: One object, food and stores

In [206]:
barcelona_merged.loc[barcelona_merged['Cluster labels'] == 3, barcelona_merged.columns[[1] + list(range(5, barcelona_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,La Marina del Prat Vermell (12),Spanish Restaurant,Supermarket,Mediterranean Restaurant,Diner,Automotive Shop,Furniture / Home Store,Insurance Office,Plaza,Food,Flea Market


Cluster 4: Restaurants, hotels, coffee shops.

In [207]:
barcelona_merged.loc[barcelona_merged['Cluster labels'] == 4, barcelona_merged.columns[[1] + list(range(5, barcelona_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,La Marina de Port (13),Spanish Restaurant,Hotel,Café,Bakery,Plaza,Restaurant,Furniture / Home Store,Latin American Restaurant,Supermarket,Kids Store
24,Provençals del Poblenou (71),Spanish Restaurant,Hotel,Café,Mediterranean Restaurant,Asian Restaurant,Convenience Store,Pedestrian Plaza,Steakhouse,Falafel Restaurant,Liquor Store
25,San Martín de Provensals (72) y La Verneda y ...,Spanish Restaurant,Soccer Field,Coffee Shop,Supermarket,Grocery Store,Farmers Market,Mediterranean Restaurant,Office,Tennis Court,Bakery
